### PROJECT 1

Docelowy notebook w którym będzie finalny kod :-)

In [ ]:
# imports
import os

# config
from source.utils.config_manager import ConfigManager

# transfomers
from source.transformers import TestTransformersFactory
from source.transformers import TrainTrasformersFactory

# data loading
from source.utils.data_loading import reorg_data
from source.utils.data_loading import get_data

In [ ]:
data_dir_path = ConfigManager.get_data_path() + 'cifar10-kaggle/'

In [ ]:
transform_train = TrainTrasformersFactory.get_train_transformer()
transform_train = TestTransformersFactory.get_test_transformer()

In [ ]:

if not os.path.exists(r"data_dir/train_valid_test"):
    reorg_data(data_dir_path, valid_ratio = 0.1)

In [ ]:
train_iter, train_valid_iter, valid_iter, test_iter = get_data(data_dir_path, transform_train, transform_train, batch_size=4)